# Building a Simple JSON API with Python and PostgreSQL

This notebook is the capstone for everything we've learned. We will build a simple but functional web API that serves JSON data directly from our PostgreSQL database. This completes the full-stack data pipeline: from database storage to application-level delivery.

We will use **Flask**, a lightweight and popular Python web framework, to create our API. This requires integrating the skills from our previous modules:

1.  Connecting to PostgreSQL with `psycopg2`.
2.  Querying `JSONB` data.
3.  Serving the results as a JSON response.

--- 
## Step 1: Install Flask

Flask is a third-party library, so we need to install it first.

In [9]:
!pip install Flask

--- 
## Step 2: The API Code

Below is the complete Python code for our simple API. We will not run this code *inside* the notebook. Instead, we will save it as a separate `.py` file and run it from the terminal.

The code does the following:
- Imports `Flask` and `psycopg2`.
- Creates a Flask application.
- Defines an API endpoint (a URL) at `/api/users/<int:user_id>`.
- When that URL is visited, it connects to PostgreSQL.
- It queries the `users_json` table (from our last notebook) for the specified `user_id`.
- It returns the user's `profile` data as a JSON response or a 404 error if the user is not found.

In [11]:
# This is the code for our 'api.py' file.
# Copy this code into a new file named api.py in the same directory as your notebook.

api_code = """\
from flask import Flask, jsonify, abort
import psycopg2
import psycopg2.extras

# --- Database Connection Details ---
DB_HOST = 'localhost'
DB_NAME = 'people'
DB_USER = 'fahad'
DB_PASS = 'secret'

# Create the Flask application
app = Flask(__name__)

# Define the API endpoint (URL route)
@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    conn = None
    try:
        # Connect to the database
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        # Use DictCursor to get dictionary-like rows
        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
            # Query for the specific user
            cur.execute("SELECT profile FROM users_json WHERE id = %s;", (user_id,))
            user_data = cur.fetchone()
        
        if user_data is None:
            # If user not found, return a 404 error
            abort(404, description=f'User with ID {user_id} not found.')
        
        # The 'profile' column is already JSONB, so user_data['profile'] is a dict
        # jsonify will convert the Python dict to a JSON response
        return jsonify(user_data['profile'])
    
    except psycopg2.Error as e:
        # Handle potential database errors
        abort(500, description=f'Database error: {e}')
    
    finally:
        if conn is not None:
            conn.close()

# This allows us to run the app directly from the script
if __name__ == '__main__':
    app.run(debug=True)
"""

# Write the code to a file named api.py
with open('api.py', 'w') as f:
    f.write(api_code)

print("API code has been written to api.py. Please proceed to the next step.")

API code has been written to api.py. Please proceed to the next step.


--- 
## Step 3: Running the API

1.  Open a new terminal or command prompt.
2.  Navigate to the same directory where you saved `api.py`.
3.  Run the following command to start the web server:

    ```bash
    python api.py
    ```

You should see output indicating that a Flask server is running, usually on `http://127.0.0.1:5000`.

--- 
## Step 4: Testing the API

With the server running, you can now test the API endpoint. Open a **new terminal** (while the server is still running in the other one) and use `curl`, or simply open the URLs in your web browser.

### Test 1: Get User 1 (Should Succeed)

In the terminal, run:
```bash
curl [http://127.0.0.1:5000/api/users/1](http://127.0.0.1:5000/api/users/1)
```
You should see the JSON profile for Fahad Shah.

### Test 2: Get User 2 (Should Succeed)

In the terminal, run:
```bash
curl [http://127.0.0.1:5000/api/users/2](http://127.0.0.1:5000/api/users/2)
```
You should see the JSON profile for Alice.

### Test 3: Get User 99 (Should Fail with 404)

In the terminal, run:
```bash
curl [http://127.0.0.1:5000/api/users/99](http://127.0.0.1:5000/api/users/99)
```
You should get a "Not Found" error, as expected.

**To stop the server, go back to its terminal window and press `Ctrl + C`.**

--- 
## Conclusion

This notebook demonstrated the final step in the data lifecycle: exposing it to other applications via a clean, simple JSON API. We have successfully:

1.  Stored complex `JSONB` data in PostgreSQL.
2.  Written a Python application to connect and query that data.
3.  Used the Flask framework to create a web server and an API endpoint.
4.  Served the database results as a standard JSON response.

This completes our journey from understanding data formats to building a functional API.